Import Libraries

In [66]:
import os
import pandas as pd
import numpy as np
import json


In [67]:
import warnings
warnings.filterwarnings("ignore")

In [68]:
categorized_questions_file = 'categorized_questions.jsonl'

# read categorized questions
categorized_questions = []
with open(categorized_questions_file, 'r') as f:
    for line in f:
        categorized_questions.append(json.loads(line))

# check if common_questions are same as categorized_questions
categorized_questions_list = [list(q.values())[0] for q in categorized_questions]

# create a dictionary to map questions to their categories
question_to_category = {q['Question']: q['Category'] for q in categorized_questions}

#### Load of the CSV Data
- CSV Data: CSV Files for Dollar Street dataset
- Dollarstreet data has 3 more countries than CVQA

In [69]:
import pandas as pd
from scipy.spatial import distance
import ast

# List all files in the directory
directory = "/home/vsl333/cultural_values/notebooks/outputs"
# exclude file ending with _peoplle_results.csv
files = [f for f in os.listdir(directory) if f.endswith('_results.csv')]
print(len(files))


2


In [70]:
files

['ds_wvs_metadata_test_True_results.csv',
 'ds_wvs_metadata_test_False_results.csv']

### Load one csv file to understand the data

In [71]:
pd.set_option('display.max_colwidth', 80)
data_country = pd.read_csv(os.path.join(directory, files[0]))
data_country.head(2)

,img_id,image_path,country,image_code,income,question_text,country_prompt,generic_prompt,option_labels,full_options,prompt,options,top10_token_prob,prob_percent_sorted,sum_prob_percent_sorted,prob_percent_keys,prob_percent_values,selection_answers
0,5d4be794cf0b3a0f3f345b34,/gpfs/work5/0/prjs0370/zhizhang/projects/cultural_values/datasets/dollarstre...,Bangladesh,ds,"tensor(175., dtype=torch.float64)",Please tell me for each of the following things how essential you think it i...,You are in a simulation game where you live in Bangladesh region. You will ...,You are in a simulation game where you live in a demographic region. You wil...,"['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']","(A) It is against democracy (spontaneous), (B) Not an essential characterist...",A chat between a curious human and an artificial intelligence assistant. The...,"['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']","[('K', 0.5195428729057312), ('J', 0.09915649890899658), ('L', 0.087505303323...","{'A': 2.93, 'B': 4.07, 'C': 2.88, 'D': 5.56, 'E': 0, 'F': 0, 'G': 3.02, 'H':...",100.00,"['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']","[2.93, 4.07, 2.88, 5.56, 0, 0, 3.02, 2.62, 0, 10.38, 54.4, 9.16, 4.98, 0]","[2.2, 0.8, 0.2, 1.0, 1.4, 2.7, 6.9, 8.4, 14.3, 18.2, 43.8, 0.0, 0.0, 0.0]"
1,5d4bec7ecf0b3a0f3f34e042,/gpfs/work5/0/prjs0370/zhizhang/projects/cultural_values/datasets/dollarstre...,Bangladesh,ds,"tensor(124.9656, dtype=torch.float64)","Generally speaking, would you say that most people can be trusted or that yo...",You are in a simulation game where you live in Bangladesh region. You will ...,You are in a simulation game where you live in a demographic region. You wil...,"['A', 'B', 'C', 'D', 'E']","(A) Most people can be trusted, (B) Need to be very careful, (C) Don't know,...",A chat between a curious human and an artificial intelligence assistant. The...,"['A', 'B', 'C', 'D', 'E']","[('D', 0.2712326645851135), ('C', 0.19083602726459503), ('E', 0.149788901209...","{'A': 11.56, 'B': 14.73, 'C': 22.99, 'D': 32.68, 'E': 18.05}",100.01,"['A', 'B', 'C', 'D', 'E']","[11.56, 14.73, 22.99, 32.68, 18.05]","[12.9, 87.0, 0.1, 0.0, 0.0]"


In [72]:
data_image = pd.read_csv(os.path.join(directory, files[1]))
data_image.head(2)

,img_id,image_path,country,image_code,income,question_text,country_prompt,generic_prompt,option_labels,full_options,prompt,options,top10_token_prob,prob_percent_sorted,sum_prob_percent_sorted,prob_percent_keys,prob_percent_values,selection_answers
0,5d4be794cf0b3a0f3f345b34,/gpfs/work5/0/prjs0370/zhizhang/projects/cultural_values/datasets/dollarstre...,Bangladesh,ds,"tensor(175., dtype=torch.float64)",Please tell me for each of the following things how essential you think it i...,You are in a simulation game where you live in Bangladesh region. You will ...,You are in a simulation game where you live in a demographic region. You wil...,"['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']","(A) It is against democracy (spontaneous), (B) Not an essential characterist...",A chat between a curious human and an artificial intelligence assistant. The...,"['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']","[('K', 0.32120469212532043), ('J', 0.12383543699979782), ('L', 0.09644313156...","{'A': 5.12, 'B': 7.22, 'C': 5.71, 'D': 8.84, 'E': 0, 'F': 0, 'G': 4.45, 'H':...",100.0,"['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N']","[5.12, 7.22, 5.71, 8.84, 0, 0, 4.45, 4.45, 0, 13.27, 34.43, 10.34, 6.17, 0]","[2.2, 0.8, 0.2, 1.0, 1.4, 2.7, 6.9, 8.4, 14.3, 18.2, 43.8, 0.0, 0.0, 0.0]"
1,5d4bec7ecf0b3a0f3f34e042,/gpfs/work5/0/prjs0370/zhizhang/projects/cultural_values/datasets/dollarstre...,Bangladesh,ds,"tensor(124.9656, dtype=torch.float64)","Generally speaking, would you say that most people can be trusted or that yo...",You are in a simulation game where you live in Bangladesh region. You will ...,You are in a simulation game where you live in a demographic region. You wil...,"['A', 'B', 'C', 'D', 'E']","(A) Most people can be trusted, (B) Need to be very careful, (C) Don't know,...",A chat between a curious human and an artificial intelligence assistant. The...,"['A', 'B', 'C', 'D', 'E']","[('C', 0.3603440821170807), ('A', 0.27628540992736816), ('B', 0.137843891978...","{'A': 28.75, 'B': 14.34, 'C': 37.49, 'D': 10.01, 'E': 9.41}",100.0,"['A', 'B', 'C', 'D', 'E']","[28.75, 14.34, 37.49, 10.01, 9.41]","[12.9, 87.0, 0.1, 0.0, 0.0]"


In [73]:
# give all unique values in question_text column
questions_country = data_country.question_text.unique()
questions_image = data_image.question_text.unique()

# check  questions which are not same?
uncommom_questions = [q for q in questions_country if q not in questions_country]
print(f"Questions which are not common in both files: {uncommom_questions}")

common_questions = [q for q in questions_country if q in questions_image]
print(f"Common questions length: {len(common_questions)}")

Questions which are not common in both files: []
Common questions length: 291


In [74]:

# jsonl_file = 'common_questions.jsonl'
# delete file if already exist
# if os.path.exists(jsonl_file):
#     os.remove(jsonl_file)
#     print(f"Deleted {jsonl_file} file")

# # save common questions as JSON Lines file
# with open(jsonl_file, 'w') as f:
#     for i, q in enumerate(common_questions):
#         json_line = json.dumps({i: q})
#         f.write(json_line + '\n')

# print(f"Saved common questions in {jsonl_file} file")


In [77]:
# add a new column 'Category' to data_1 and data_2 based on the question_text
# set width of pandas
data_country['question_topic'] = data_country['question_text'].map(question_to_category)
data_image['question_topic'] = data_image['question_text'].map(question_to_category)

pd.set_option('display.max_colwidth', 300)
# only dsiplay question_text and question_topic columns
data_image[['question_text', 'question_topic']].tail(5)


,question_text,question_topic
15255,On this list are various groups of people. Could you please mention any that you would not like to have as neighbors?\n\nHomosexuals,I. Gender and LGBTQ
15256,On this list are various groups of people. Could you please mention any that you would not like to have as neighbors?\n\nHomosexuals,I. Gender and LGBTQ
15257,"I am going to name a number of organizations. For each one, could you tell me how much confidence you have in them: is it a great deal of confidence, quite a lot of confidence, not very much confidence or none at all?\n\nThe government (in your nation’s capital)",D. Politics and policy
15258,"For each of the following aspects, indicate how important it is in your life. Would you say it is very important, rather important, not very important or not important at all\n\nPolitics",D. Politics and policy
15259,"Here are two statements people sometimes make when discussing the environment and economic growth. Which of them comes closer to your own point of view?\n\nA. Protecting the environment should be given priority, even if it causes slower economic growth and some loss of jobs\n\nB. Economic growth...",N. Economy and work


In [86]:
results = []

# loop over data_image and data_country
for idx, each_data in enumerate([data_country, data_image]):
    data = each_data

    # Add category column to the DataFrame
    # data['category'] = data['image_path'].apply(lambda x: x.split('/')[-1].split('.')[0].split('_')[1:-2])
    # data['category'] = data['category'].apply(lambda x: '_'.join(x))

    # Example gt and pred (list of lists)
    gt = [ast.literal_eval(x) for x in data['selection_answers'].tolist()] # there are prob distrbutions of human answers
    pred = [ast.literal_eval(x) for x in data['prob_percent_values'].tolist()] # there are prob distrbutions of model answers

    # Original DataFrame, retaining all columns from `data`
    df = data.copy()

    # Keep track of matching indices
    matching_indices = [i for i, (g, p) in enumerate(zip(gt, pred)) if len(g) == len(p)]

    # Filter rows where len(g) == len(p)
    filtered_gt_pred = [(g, p) for g, p in zip(gt, pred) if len(g) == len(p)]

    # If there are any matching rows
    if filtered_gt_pred:
        gt_filtered, pred_filtered = zip(*filtered_gt_pred)

        # Calculate JSD
        jsd = [distance.jensenshannon(g, p) for g, p in zip(gt_filtered, pred_filtered)]
        similarity = [1 - value for value in jsd]

        # Add JSD and similarity values back to the DataFrame
        df.loc[matching_indices, 'jsd'] = jsd
        df.loc[matching_indices, 'similarity'] = similarity
    else:
        df['jsd'] = None  # or handle the case accordingly
        df['similarity'] = None

    # Set JSD and similarity to NaN or a placeholder for rows that were filtered out
    df['jsd'] = pd.to_numeric(df['jsd'], errors='coerce')
    df['similarity'] = pd.to_numeric(df['similarity'], errors='coerce')
    if idx ==0:
        df['Image'] = 'False'
    else:
        df['Image'] = 'True'

    #
    # Calculate mean of jsd and similarity per country per 'Image' column and add new col 'mean_jsd' and 'mean_similarity'
    df['country_mean_jsd'] = df.groupby(['country', 'Image'])['jsd'].transform('mean')
    df['coutry_mean_similarity'] = df.groupby(['country', 'Image'])['similarity'].transform('mean')
    # mean_per_country = df.groupby(['country', 'Image'])[['jsd', 'similarity']].mean().reset_index()
    # mean_per_country = df.groupby('country')[['jsd', 'similarity']].mean().reset_index()

    # Append results to list
    results.append(df)

# Combine all results into a single DataFrame
# final_result = pd.concat(results, ignore_index=True)
final_result = pd.concat(results)

In [107]:
pd.set_option('display.max_colwidth', 50)
final_result.tail(2)

,img_id,image_path,country,image_code,income,question_text,country_prompt,generic_prompt,option_labels,full_options,prompt,options,top10_token_prob,prob_percent_sorted,sum_prob_percent_sorted,prob_percent_keys,prob_percent_values,selection_answers,question_topic,jsd,similarity,Image,country_mean_jsd,coutry_mean_similarity
15258,5d4becfbcf0b3a0f3f34ece6,/gpfs/work5/0/prjs0370/zhizhang/projects/cultu...,Vietnam,ds,"tensor(265.5254, dtype=torch.float64)","For each of the following aspects, indicate ho...",You are in a simulation game where you live i...,You are in a simulation game where you live in...,"['A', 'B', 'C', 'D', 'E', 'F', 'G']","(A) Very important, (B) Rather important, (C) ...",A chat between a curious human and an artifici...,"['A', 'B', 'C', 'D', 'E', 'F', 'G']","[('D', 0.4119711220264435), ('C', 0.2239848077...","{'A': 15.9, 'B': 8.38, 'C': 22.42, 'D': 41.25,...",100.00,"['A', 'B', 'C', 'D', 'E', 'F', 'G']","[15.9, 8.38, 22.42, 41.25, 6.53, 4.02, 1.5]","[8.9, 31.2, 47.2, 12.7, 0.0, 0.0, 0.0]",D. Politics and policy,0.379592,0.620408,True,0.499283,0.500717
15259,5d4bed0fcf0b3a0f3f34eee2,/gpfs/work5/0/prjs0370/zhizhang/projects/cultu...,Vietnam,ds,"tensor(245.9814, dtype=torch.float64)",Here are two statements people sometimes make ...,You are in a simulation game where you live i...,You are in a simulation game where you live in...,"['A', 'B', 'C', 'D', 'E', 'F']","(A) Protecting environment, (B) Economy growth...",A chat between a curious human and an artifici...,"['A', 'B', 'C', 'D', 'E', 'F']","[('A', 0.650934636592865), ('C', 0.13432794809...","{'A': 65.11, 'B': 10.14, 'C': 13.44, 'D': 10.1...",100.01,"['A', 'B', 'C', 'D', 'E', 'F']","[65.11, 10.14, 13.44, 10.14, 0.75, 0.43]","[71.7, 26.2, 2.1, 0.1, 0.0, 0.0]",N. Economy and work,0.281009,0.718991,True,0.499283,0.500717


### For each category, group by country, jsd and 

In [120]:
# for each question_topic, calculate mean of jsd and similarity per country and save as different df 
# save all dfs in a dictionary

import shutil
category_outputs_dir = 'category_outputs'

# delete directory if already exist
if os.path.exists(category_outputs_dir):
    shutil.rmtree(category_outputs_dir)
    print(f"Deleted {category_outputs_dir} directory")

# create directory
os.makedirs(category_outputs_dir)
print(f"Created {category_outputs_dir} directory")  

# save final_result as csv
final_result.to_csv(f"{category_outputs_dir}/all_category_results.csv", index=False)

df_category_similarity = {}

for category in final_result['question_topic'].unique():
    df_category = final_result[final_result['question_topic'] == category]

    # calculate mean of jsd and similarity per country
    df_category['category_mean_jsd'] = df_category.groupby(['country', 'Image'])['jsd'].transform('mean')
    df_category['category_mean_similarity'] = df_category.groupby(['country', 'Image'])['similarity'].transform('mean')

    df_category_similarity[category] = df_category

    df_category_similarity[category].to_csv(f"{category_outputs_dir}/{category}_results.csv", index=False)



Deleted category_outputs directory


In [108]:
df_category_similarity.keys()

dict_keys(['P. Methodological research', 'D. Politics and policy', 'N. Economy and work', 'L. Family and relationships', 'J. News habits and media', 'O. Regions and countries', 'Q. Security', 'B. Religion and spirituality', 'C. Science and technology', 'K. Immigration and migration', 'E. Demographics', 'G. International affairs', 'A. Social values and attitudes', 'I. Gender and LGBTQ', 'M. Race and ethnicity'])

In [115]:
# for  key, show country_mean_jsd, country_mean_similarity per country
cat_df = df_category_similarity['D. Politics and policy']

questions = cat_df['question_text'].unique()

# print for frist question
question = questions[0]

# 5d4bed0fcf0b3a0f3f34eee2, 5d4bf4b0cf0b3a0f3f35c062
cat_df[(cat_df['question_text'] == question) & (cat_df['country'] == 'Vietnam') & (cat_df['img_id'] == '5d4bed0fcf0b3a0f3f34eee2')]

,img_id,image_path,country,image_code,income,question_text,country_prompt,generic_prompt,option_labels,full_options,prompt,options,top10_token_prob,prob_percent_sorted,sum_prob_percent_sorted,prob_percent_keys,prob_percent_values,selection_answers,question_topic,jsd,similarity,Image,country_mean_jsd,coutry_mean_similarity,category_mean_jsd,category_mean_similarity
14426,5d4bed0fcf0b3a0f3f34eee2,/gpfs/work5/0/prjs0370/zhizhang/projects/cultu...,Vietnam,ds,"tensor(245.9814, dtype=torch.float64)",Now I'd like you to look at this card. I'm goi...,You are in a simulation game where you live i...,You are in a simulation game where you live in...,"['A', 'B', 'C', 'D', 'E', 'F']","(A) Have done, (B) Might do, (C) Would never d...",A chat between a curious human and an artifici...,"['A', 'B', 'C', 'D', 'E', 'F']","[('D', 0.30740606784820557), ('A', 0.286533951...","{'A': 29.1, 'B': 9.9, 'C': 14.18, 'D': 31.22, ...",99.98,"['A', 'B', 'C', 'D', 'E', 'F']","[29.1, 9.9, 14.18, 31.22, 8.67, 6.91]","[3.9, 20.5, 75.6, 0.0, 0.0, 0.0]",D. Politics and policy,0.584307,0.415693,False,0.500647,0.499353,0.47163,0.52837
14426,5d4bed0fcf0b3a0f3f34eee2,/gpfs/work5/0/prjs0370/zhizhang/projects/cultu...,Vietnam,ds,"tensor(245.9814, dtype=torch.float64)",Now I'd like you to look at this card. I'm goi...,You are in a simulation game where you live i...,You are in a simulation game where you live in...,"['A', 'B', 'C', 'D', 'E', 'F']","(A) Have done, (B) Might do, (C) Would never d...",A chat between a curious human and an artifici...,"['A', 'B', 'C', 'D', 'E', 'F']","[('C', 0.3023563623428345), ('D', 0.2840375006...","{'A': 15.01, 'B': 18.39, 'C': 30.31, 'D': 28.4...",100.00,"['A', 'B', 'C', 'D', 'E', 'F']","[15.01, 18.39, 30.31, 28.48, 4.95, 2.86]","[3.9, 20.5, 75.6, 0.0, 0.0, 0.0]",D. Politics and policy,0.439874,0.560126,True,0.499283,0.500717,0.47163,0.52837


In [117]:
# for  key, show country_mean_jsd, country_mean_similarity per country
cat_df = df_category_similarity['D. Politics and policy']

# filter country_mean_jsd and country_mean_similarity columns, country and Image columns. then drop duplicates
cat_df = cat_df[['country', 'category_mean_jsd', 'category_mean_similarity', 'Image']].drop_duplicates()
# sort by Country and Image
cat_df = cat_df.sort_values(['country', 'Image'])
cat_df

,country,category_mean_jsd,category_mean_similarity,Image
2,Bangladesh,0.395896,0.604104,False
2,Bangladesh,0.386972,0.613028,True
1282,Brazil,0.401873,0.598127,False
1282,Brazil,0.375270,0.624730,True
2778,China,0.467166,0.532834,False
2778,China,0.472857,0.527143,True
6855,France,0.404899,0.595101,False
6855,France,0.371557,0.628443,True
6974,Iran,0.421431,0.578569,False
6974,Iran,0.417853,0.582147,True


In [24]:
#sort final_result by country name and then similarity
final_result = final_result.sort_values(by=['similarity'], ascending=True)
final_result.head(60)

,country,Image,jsd,similarity
24,South Korea,True,0.513882,0.486118
10,South Korea,False,0.512263,0.487737
13,Vietnam,False,0.500647,0.499353
27,Vietnam,True,0.499283,0.500717
16,China,True,0.494845,0.505155
2,China,False,0.486084,0.513916
4,Iran,False,0.473466,0.526534
14,Bangladesh,True,0.466264,0.533736
0,Bangladesh,False,0.460290,0.539710
12,United States,False,0.457596,0.542404
